In [1]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import learning_curve
from sklearn.decomposition import PCA

# `Load extracted feature maps`

In [ ]:
X_train = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/train_feature_maps.npy')
X_test = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/test_feature_maps.npy')
X_val = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/val_feature_maps.npy')
y_train = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/train_labels.npy')
y_test = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/test_labels.npy')
y_val = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/val_labels.npy')

# `Convert one-hot encoded labels to class indices`

In [ ]:
y_train_labels = np.argmax(y_train, axis=1)
y_test_labels = np.argmax(y_test, axis=1)
y_val_labels = np.argmax(y_val, axis=1)

In [3]:
# Reshape 4D feature maps to 2D (samples, features)
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)
X_val = X_val.reshape(X_val.shape[0], -1)

# `Standardize features`

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

# `Apply PCA for dimensionality reduction`

In [ ]:
pca = PCA(n_components=500)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
X_val_pca = pca.transform(X_val_scaled)

# `Naïve Bayes`

In [ ]:
dt_model = DecisionTreeClassifier(criterion='gini', max_depth=100, random_state=42)
dt_model.fit(X_train_pca, y_train_labels)

DecisionTreeClassifier(max_depth=100, random_state=42)

# `Save models`

In [ ]:
joblib.dump(dt_model, "/kaggle/working/dt.pkl")
joblib.dump(scaler, "/kaggle/working/scaler.pkl")
joblib.dump(pca, "/kaggle/working/pca.pkl")

['/kaggle/working/pca.pkl']

# `Testing performance`

In [8]:
y_train_pred = dt_model.predict(X_train_pca)
y_val_pred = dt_model.predict(X_val_pca)
y_test_pred = dt_model.predict(X_test_pca)

In [9]:
# Compute accuracy for train, validation, and test sets
train_accuracy = accuracy_score(y_train_labels, y_train_pred)
val_accuracy = accuracy_score(y_val_labels, y_val_pred)
test_accuracy = accuracy_score(y_test_labels, y_test_pred)

In [10]:
# Print accuracies
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Training Accuracy: 29.80%
Validation Accuracy: 10.71%
Test Accuracy: 12.81%
